In [51]:
!vec-inf launch llama-3.1 --model-variant="8B-Instruct" --qos="normal"

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Job Config    ┃ Value                      ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Slurm Job ID  │ 13403958                   │
│ Job Name      │ Meta-Llama-3.1-8B-Instruct │
│ Partition     │ a40                        │
│ Num Nodes     │ 1                          │
│ GPUs per Node │ 1                          │
│ QOS           │ normal                     │
│ Walltime      │ 04:00:00                   │
│ Data Type     │ auto                       │
└───────────────┴────────────────────────────┘


In [10]:
from openai import OpenAI
client = OpenAI(base_url="http://gpu003:8080/v1", api_key="EMPTY")

DATA_ROOT = '/datasets/PMC-15M'
SAMPLE_DATA = f'{DATA_ROOT}/0.jsonl'

# Load sample jsonl file
with open(SAMPLE_DATA, 'r') as f:
    dataset = [eval(line) for line in f]

print(f"Dataset size: {len(dataset)}"); dataset[23]

Dataset size: 9568


{'PMC_ID': 'PMC193607',
 'media_id': 'pbio-0000031-g001',
 'caption': 'Electron micrograph of Proteobacteria in eukaryotic cell',
 'media_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC193607/bin/pbio.0000031.g001.jpg',
 'media_name': 'PMC193607_pbio-0000031-g001.jpg'}

In [295]:
SYSTEM_PROMPT = """
Subfigure labels are letters referring to individual subfigures within a larger figure.
Check if the caption contains explicit subfigure label. 
If not or there is only one subfigure, output "NO" and end the generation. 
If yes, output "YES", then generate the subcaption of the subfigures according to the caption. 
The output should use the template: Yes.\n Subfigure-A: ... \n Subfigure-B: ... \n ......
The label should be removed from subcaption.
""".strip()

# ================================================================================================================================================================================== #

# SYSTEM_PROMPT = """
# You are an expert in analyzing figure captions, breaking them into subcaptions.
# - Determine if the caption contains excplicit labeled subfigures (A, B, ...).
# - If one or zero subfigures, respond with "NO" and exit generation.
# - If more than one subfigures exist, respond with "YES" followed by:
#     Subfigure-A: [Content for A, excluding label]
#     Subfigure-B: [Content for B, excluding label]
#     [Continue for additional subfigures]
# - If there was only one or zero subfigures, end with "NONE".
# """.strip()

caption = dataset[1112]['caption']  #116 + 1002 #2575 #2100+3821 #2100+1015 #6125 #9476 #123 #4721 #2512 #5555 #12

USER_PROMPT = f"""
Caption: \n{caption}
""".strip()

print(f"Length of prompt: {len(SYSTEM_PROMPT.split(' '))}, Length of caption: {len(caption.split(' '))}\n\n {caption}")

Length of prompt: 68, Length of caption: 272

 Affinity Purification of Emerin-Associated Proteins(A) Immunoblot of HeLa nuclear lysate proteins (L), or proteins affinity-purified using either BSA beads or emerin beads (see Materials and Methods), probed with antibody against actin.(B) HeLa nuclear lysate proteins (L) were immunoprecipitated using either immune (Im) or preimmune (PI) serum 2999 against emerin, resolved by SDS-PAGE, and Western blotted using antibodies specific for actin (upper panel) or emerin (lower panel), in succession.(C) Cosedimentation assays using F-actin and purified, recombinant wild-type emerin (residues 1–222). G-actin (2 μM) was polymerized and then incubated in the absence or presence of 2 μM emerin. Emerin was incubated alone in polymerization buffer as a negative control. After 30 min samples were centrifuged 1 h at 100,000g, resolved by SDS-PAGE, and stained with Coomassie blue. L, load (100%); S, supernatant (100%); P, pellet (100%).(D) F-actin column 

In [296]:
completion = client.chat.completions.create(
  model="/model-weights/Meta-Llama-3.1-8B-Instruct",
  messages=[
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
  ],
  temperature=0,
  max_tokens=500,
)

print(completion.choices[0].message.content)

YES.
 Subfigure-A: Immunoblot of HeLa nuclear lysate proteins (L), or proteins affinity-purified using either BSA beads or emerin beads (see Materials and Methods), probed with antibody against actin.
 Subfigure-B: HeLa nuclear lysate proteins (L) were immunoprecipitated using either immune (Im) or preimmune (PI) serum 2999 against emerin, resolved by SDS-PAGE, and Western blotted using antibodies specific for actin (upper panel) or emerin (lower panel), in succession.
 Subfigure-C: Cosedimentation assays using F-actin and purified, recombinant wild-type emerin (residues 1–222). G-actin (2 μM) was polymerized and then incubated in the absence or presence of 2 μM emerin. Emerin was incubated alone in polymerization buffer as a negative control. After 30 min samples were centrifuged 1 h at 100,000g, resolved by SDS-PAGE, and stained with Coomassie blue. L, load (100%); S, supernatant (100%); P, pellet (100%).
 Subfigure-D: F-actin column was used to determine the affinity of F-actin for 